# Final Project - NLP data prep

## Importing packages and data

In [1]:
import base64
import numpy as np
import pandas as pd
import os

# Plotly imports
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.express as px
from PIL import Image


# Other imports
from collections import Counter
import requests
import re
from nltk.corpus import stopwords
import string
import nltk
import imageio
from wordcloud import WordCloud, STOPWORDS
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from matplotlib import pyplot as plt

In [2]:
filepath = 'C:/Users/User/Documents/Data Science/Final Project/nlp_df.csv'

In [3]:
ncm_df = pd.read_csv(filepath)

In [4]:
ncm_df.sample(100)

,Unnamed: 0,Complete description for Import Declaration,NCM Code
3261,3292,Placa de aço carbono com a função de içamento ...,73269090
532,562,Quadro de aço carbono para aplicação no filtro...,73269090
2847,2877,Tubulação em aço liga normalizado sem costura ...,73045119
3228,3259,Tampa com a função de batente de segurança par...,73269090
209,238,Válvula borboleta com volante e engrenagem par...,84818097
...,...,...,...
146,173,Conjunto de elevação e movimentação de cargas ...,84289090
1025,1055,Redução concêntrica para solda em aço ASTM A23...,73079300
814,844,Acessório em T de 90º de Polipropileno (PPH) ...,39174090
2463,2493,Estrutura de suporte; Material: Aço carbono,73269090


In [5]:
ncm_df.drop('Unnamed: 0', axis=1, inplace=True)

In [6]:
ncm_df

,Complete description for Import Declaration,NCM Code
0,Conjunto do bloco do queimador,84179000
1,Suporte do tubo de resfriamento,84179000
2,Suporte para isolamento 250mm,84179000
3,"Tubo de aço inoxidável de 9,3 mm (SCH 40S X 30...",73042400
4,"Tubo de aço inoxidável de 12,7 mm (SCH 40S X 6...",73042400
...,...,...
3345,"EX 081 Fornos industriais, com aquecimento a g...",84178090
3346,Máquinas laminadora para encruamento de chapas...,84552290
3347,EX 210 Equipamentos para controle da quantidad...,84798999
3348,"EX 081 Fornos industriais, com aquecimento a g...",84178090


In [7]:
ncm_df['NCM Code'].value_counts().sort_values().head(50)

84287000    1
84798190    1
90329099    1
84283200    1
85068090    1
85015290    1
72163200    1
84716053    1
72166190    1
73045190    1
39209910    1
59061000    1
85051990    1
85015210    1
85371020    1
84439931    1
85015310    1
73144100    1
84619090    1
84714100    1
84798911    1
85362000    1
38160019    1
81039900    1
72162100    1
85234990    1
84249090    1
84811000    1
84193900    1
82079000    1
40103500    1
73151100    1
39269022    1
84139190    1
84798290    1
39269021    1
84733090    1
85052090    1
84818099    1
84195090    1
84243090    1
72286000    1
35069110    1
73130000    1
70198000    1
82059000    1
85369010    1
85015390    2
84292090    2
84821010    2
Name: NCM Code, dtype: int64

## NLP processing

In [8]:
from nltk.tokenize import word_tokenize

In [9]:
def clean_tokens(text,language):
    # 1 create tokens
    tokens = word_tokenize(text)
    # 2 lower case
    tokens = [w.lower() for w in tokens]
    # 3 remove punctuations
    stripped = [word for word in tokens if word.isalpha()]
    # 4 remove stop_words
    stop_words = set(stopwords.words(language))
    words = [w for w in stripped if not w in stop_words]
    # return 
    return words

In [10]:
ncm_df['clean_description'] = ncm_df['Complete description for Import Declaration'].apply(lambda x: clean_tokens(x,'portuguese'))

In [32]:
ncm_df[200:205]

,Complete description for Import Declaration,NCM Code,clean_description
200,Válvula de retenção para sistema de lubrificação,84813000,válvula retenção sistema lubrificação
201,Trocador de calor de placas,84195010,trocador calor placas
202,Válvulas solenóide de duas vias,84818092,válvulas solenóide duas vias
203,Arruelas de vedação de plástico pra aplicação ...,39269010,arruelas vedação plástico pra aplicação tubos
204,Válvula borboleta com volante e engrenagem par...,84818097,válvula borboleta volante engrenagem sistema l...


In [12]:
# After preprocessing, the text format
def combine_text(list_of_text):
    '''Takes a list of text and combines them into 
    one large chunk of text.'''
    combined_text = ' '.join(list_of_text)
    return combined_text

In [13]:
ncm_df['clean_description'] = ncm_df['clean_description'].apply(lambda x : combine_text(x))
ncm_df.head(100)

,Complete description for Import Declaration,NCM Code,clean_description
0,Conjunto do bloco do queimador,84179000,conjunto bloco queimador
1,Suporte do tubo de resfriamento,84179000,suporte tubo resfriamento
2,Suporte para isolamento 250mm,84179000,suporte isolamento
3,"Tubo de aço inoxidável de 9,3 mm (SCH 40S X 30...",73042400,tubo aço inoxidável mm sch x forno galvanizaçã...
4,"Tubo de aço inoxidável de 12,7 mm (SCH 40S X 6...",73042400,tubo aço inoxidável mm sch x forno galvanizaça...
...,...,...,...
95,Junta de expansão para absorber vibrações em t...,84842000,junta expansão absorber vibrações tubulação fa...
96,Junta de expansão para absorber vibrações em t...,84842000,junta expansão absorber vibrações tubulação fa...
97,Junta de expansão para absorber vibrações em t...,84842000,junta expansão absorber vibrações tubulação fa...
98,Compensador de dilatação para absorver variaçã...,84849000,compensador dilatação absorver variação dimens...


## TFIDF

In [14]:
tfidf = TfidfVectorizer()

sparse_matrix = tfidf.fit_transform(ncm_df['clean_description'])

sparse_matrix

<3350x1672 sparse matrix of type '<class 'numpy.float64'>'
	with 31311 stored elements in Compressed Sparse Row format>

In [15]:
sparse_matrix.todense()

matrix([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.17287724,
         0.        ]])

In [16]:
from sklearn.neighbors import NearestNeighbors
kNN = NearestNeighbors(n_neighbors=10,  metric='cosine')
kNN
kNN.fit(sparse_matrix)

NearestNeighbors(metric='cosine', n_neighbors=10)

In [17]:
def recommended_ncms(df, id, k, sparse_matrix, metric='cosine'):

    ncm = df.loc[id,['clean_description', 'NCM Code']]
    ncm_to_assess = sparse_matrix[id]
    
    neighbour_ids = []
    
    kNN = NearestNeighbors(n_neighbors=k, 
                           metric=metric)

    kNN.fit(sparse_matrix)
    
    neighbour = kNN.kneighbors(ncm_to_assess, return_distance=False)

    # We need to map each neighbour id with the right recipe
    for i in range(1,k):
        n = neighbour.item(i) #exctracting the 
        neighbour_ids.append(n)

    print("Because the description is: {}, similar items are".format(ncm))
    print('-----------------------------------------------')
    print(df.loc[neighbour_ids, ['NCM Code', 'clean_description']])

In [18]:
recommended_ncms(ncm_df, 5, 10, sparse_matrix, metric='cosine')

Because the description is: clean_description    tubo aço inoxidável sch x forno galvanização i...
NCM Code                                                      73042400
Name: 5, dtype: object, similar items are
-----------------------------------------------
    NCM Code                                  clean_description
5   73042400  tubo aço inoxidável sch x forno galvanização i...
10  73042400  tubo aço inoxidável sch x forno galvanização i...
16  73042400  tubo aço inoxidável sch x forno galvanização i...
8   73042400  tubo aço inoxidável sch x forno galvanização i...
13  73042400  tubo aço inoxidável sch x forno galvanização i...
9   73042400  tubo aço inoxidável sch x forno galvanização i...
14  73042400  tubo aço inoxidável sch x forno galvanização i...
6   73042400  tubo aço inoxidável mm sch x forno galvanizaçã...
3   73042400  tubo aço inoxidável mm sch x forno galvanizaçã...


## Testing and Tuning

In [19]:
#Selecting a sample from the original df
ncm_test = ncm_df.sample(1000, random_state=42)

In [20]:
ncm_test.head()

,Complete description for Import Declaration,NCM Code,clean_description
1830,Arruela de aço carbono ISO 7089 ST-HV200; Apli...,73182200,arruela aço carbono iso aplicação instalação c...
2628,Coluna de suporte em aço carbono; Aplicação: S...,73269090,coluna suporte aço carbono aplicação suporte s...
1552,Arruela de pressão para aplicação em dispositivo,73182100,arruela pressão aplicação dispositivo
1236,Eixo em aço ligado normalizado S355J2+N para a...,73269090,eixo aço ligado normalizado aplicação bloco fl...
3079,"Cone de solda 24° em aço carbono galvanizado, ...",73079900,cone solda aço carbono galvanizado porca rosca...


In [21]:
ncm_test_i = ncm_test.reset_index()

In [65]:
kNN = NearestNeighbors(n_neighbors=10, 
                           metric='cosine')

kNN.fit(sparse_matrix)

NearestNeighbors(metric='cosine', n_neighbors=10)

In [66]:
def recommended_test(df, id, sparse_matrix, model):

    ncm = df.loc[id,['clean_description', 'NCM Code']]
    ncm_to_assess = sparse_matrix[id]
    
    neighbour_ids = []

    neighbour = model.kneighbors(ncm_to_assess, return_distance=False)

    # We need to map each neighbour id with the right recipe
    for i in range(1,10):
        n = neighbour.item(i) #exctracting the 
        neighbour_ids.append(n)

    flat_list = [item for sublist in df.loc[neighbour_ids, ['NCM Code']].values for item in sublist]
    
    return flat_list


In [67]:
ncm_test_i['recommended_ncms'] = ncm_test_i['index'].apply(lambda x: 
                                                       recommended_test(ncm_df, x, sparse_matrix, kNN))

In [68]:
ncm_test_i

,index,Complete description for Import Declaration,NCM Code,clean_description,recommended_ncms,code_score,most_common_code
0,1830,Arruela de aço carbono ISO 7089 ST-HV200; Apli...,73182200,arruela aço carbono iso aplicação instalação c...,"[73182200, 73182200, 73182200, 73182200, 73182...",1.000,73182200
1,2628,Coluna de suporte em aço carbono; Aplicação: S...,73269090,coluna suporte aço carbono aplicação suporte s...,"[73269090, 73269090, 73269090, 73269090, 73269...",1.000,73269090
2,1552,Arruela de pressão para aplicação em dispositivo,73182100,arruela pressão aplicação dispositivo,"[73182100, 73182100, 73181500, 84798912, 73182...",0.375,73182100
3,1236,Eixo em aço ligado normalizado S355J2+N para a...,73269090,eixo aço ligado normalizado aplicação bloco fl...,"[73269090, 73269090, 84123110, 84123110, 73181...",0.250,73181500
4,3079,"Cone de solda 24° em aço carbono galvanizado, ...",73079900,cone solda aço carbono galvanizado porca rosca...,"[73079900, 73079900, 73079900, 73079900, 73079...",1.000,73079900
...,...,...,...,...,...,...,...
995,2002,Ancoragem de pino roscado em aço carbono zinca...,73181500,ancoragem pino roscado aço carbono zincado apl...,"[73181500, 73181500, 73181500, 73181500, 73181...",1.000,73181500
996,2472,Proteção de tiras para equipamento; Material: ...,73269090,proteção tiras equipamento material aço carbono,"[73269090, 73269090, 73269090, 73269090, 73269...",1.000,73269090
997,2954,"Conector giratório de conexão roscada, em aço ...",73079200,conector giratório conexão roscada aço inoxidá...,"[73079200, 73079200, 73079200, 73079200, 73079...",1.000,73079200
998,518,Soquete em aço carbono soldado para aplicação...,73079300,soquete aço carbono soldado aplicação tubulaçõ...,"[73079100, 73079100, 73079100, 73079100, 73079...",0.125,73079100


In [69]:
from statistics import mean

In [70]:
result = []
for i, ncm in enumerate(ncm_test_i['NCM Code']):
    
    if ncm in ncm_test_i.loc[i, 'recommended_ncms'] :
        result.append(1)
    else:
        result.append(0)
print(mean(result))

0.949


In [71]:
def this_is_the_func(row):
    total_count = row['recommended_ncms'].count(row['NCM Code'])
    score = total_count/len(row['recommended_ncms'])
    
    return score

In [72]:
ncm_test_i["code_score"] = ncm_test_i.apply(this_is_the_func, axis=1)

In [73]:
ncm_test_i["code_score"].mean()

0.7484444444444445

In [74]:
#Out of the 10 closest neighbors to each code, 75% of the time the recommender will give the same code

In [75]:
from scipy import stats as s

def find_most_common_code(row):
     return int(s.mode(row['recommended_ncms'])[0])

ncm_test_i['most_common_code'] = ncm_test_i.apply(find_most_common_code, axis=1)

In [76]:
ncm_test_i

,index,Complete description for Import Declaration,NCM Code,clean_description,recommended_ncms,code_score,most_common_code
0,1830,Arruela de aço carbono ISO 7089 ST-HV200; Apli...,73182200,arruela aço carbono iso aplicação instalação c...,"[73182200, 73182200, 73182200, 73182200, 73182...",1.000000,73182200
1,2628,Coluna de suporte em aço carbono; Aplicação: S...,73269090,coluna suporte aço carbono aplicação suporte s...,"[73269090, 73269090, 73269090, 73269090, 73269...",1.000000,73269090
2,1552,Arruela de pressão para aplicação em dispositivo,73182100,arruela pressão aplicação dispositivo,"[73182100, 73182100, 73181500, 84798912, 73182...",0.333333,73182100
3,1236,Eixo em aço ligado normalizado S355J2+N para a...,73269090,eixo aço ligado normalizado aplicação bloco fl...,"[73269090, 73269090, 84123110, 84123110, 73181...",0.333333,73181500
4,3079,"Cone de solda 24° em aço carbono galvanizado, ...",73079900,cone solda aço carbono galvanizado porca rosca...,"[73079900, 73079900, 73079900, 73079900, 73079...",1.000000,73079900
...,...,...,...,...,...,...,...
995,2002,Ancoragem de pino roscado em aço carbono zinca...,73181500,ancoragem pino roscado aço carbono zincado apl...,"[73181500, 73181500, 73181500, 73181500, 73181...",1.000000,73181500
996,2472,Proteção de tiras para equipamento; Material: ...,73269090,proteção tiras equipamento material aço carbono,"[73269090, 73269090, 73269090, 73269090, 73269...",0.888889,73269090
997,2954,"Conector giratório de conexão roscada, em aço ...",73079200,conector giratório conexão roscada aço inoxidá...,"[73079200, 73079200, 73079200, 73079200, 73079...",1.000000,73079200
998,518,Soquete em aço carbono soldado para aplicação...,73079300,soquete aço carbono soldado aplicação tubulaçõ...,"[73079100, 73079100, 73079100, 73079100, 73079...",0.222222,73079100


In [77]:
result2 = []
for i, ncm in enumerate(ncm_test_i['NCM Code']):
    
    if ncm == ncm_test_i.loc[i, 'most_common_code'] :
        result2.append(1)
    else:
        result2.append(0)
print(mean(result2))

0.784
